In [ ]:
#@markdown Qut-Noise experements (not available yet)
# !mkdir data

# main?
# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/8342a090-89e7-4402-961e-1851da11e1aa/download/qutnoise.zip
# !unzip qutnoise.zip -d data/ > /dev/null

# car
# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/7412452a-92e9-4612-9d9a-6b00f167dc15/download/qutnoisecar.zip
# !unzip qutnoisecar.zip -d data/ > /dev/null

# cafe
# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/9b0f10ed-e3f5-40e7-b503-73c2943abfb1/download/qutnoisecafe.zip
# !unzip qutnoisecafe.zip -d data/

# home
# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/35cd737a-e6ad-4173-9aee-a1768e864532/download/qutnoisehome.zip
# !unzip qutnoisehome.zip -d data/

# reverb?
# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/164d38a5-c08e-4e20-8272-793534eb10c7/download/qutnoisereverb.zip
# !unzip qutnoisereverb.zip -d data/

# street
# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/10eeceae-9f0c-4556-b33a-dcf35c4f4db9/download/qutnoisestreet.zip
# !unzip qutnoisestreet.zip -d data/


In [1]:
#@markdown Loading weights & testing samples (from ur Google Drive)
!mkdir results
!mkdir samples
!mkdir samples/noisy

from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/train_vb.ckpt' '/content/'
!cp '/content/gdrive/My Drive/sample_0.wav' '/content/samples/noisy/'

Mounted at /content/gdrive


In [ ]:
# @markdown Model sgmse prep
!git clone https://github.com/sp-uhh/sgmse.git > /dev/null
# !cd sgmse && git checkout cc3518f5ba06ca8f046a497b6ef3408a54ae8399


# !wget -O model_vb_epo115.cpkt https://drive.google.com/file/d/1_H3EXvhcYBhOZ9QNUcD5VZHc6ktrRbwQ/view?usp=share_link
# !wget -O wsj.cpkt https://drive.google.com/file/d/16K4DUdpmLhDNC7pJhBBc08pkSIn_yMPi/view?usp=share_link
# !pip install -r sgmse/requirements.txt > /dev/null

In [ ]:
# @markdown DEMAND Dataset prep
!mkdir data
!wget -O testset.zip https://datashare.ed.ac.uk/bitstream/handle/10283/1942/noisy_testset_wav.zip?sequence=10&isAllowed=y
!unzip testset.zip -d data > /dev/null

!mkdir samples
!cp data/noisy_testset_wav/p232_001.wav samples/sample_0.wav
!cp data/noisy_testset_wav/p232_002.wav samples/sample_1.wav
!cp data/noisy_testset_wav/p232_003.wav samples/sample_2.wav

In [3]:
#@markdown Installing requirements
# !pip install protobuf==3.19.4
# !pip install setuptools==59.5.0
# !pip install pytorch-lightning > /dev/null

!pip install pytorch-lightning==1.6.5 > /dev/null
!pip install torch-ema==0.3 > /dev/null
!pip install Ninja==1.10.2.3 > /dev/null
!pip install pesq==0.0.4 > /dev/null
!pip install pystoi==0.3.3 > /dev/null

In [4]:
#@markdown Inference
!python3 sgmse/enhancement.py --test_dir samples --enhanced_dir results --ckpt train_vb.ckpt

100% 1/1 [00:27<00:00, 27.58s/it]


In [5]:
import IPython

IPython.display.display(IPython.display.Audio('samples/noisy/sample_0.wav'))
IPython.display.display(IPython.display.Audio('results/sample_0.wav'))

In [ ]:
import glob
# from argparse import ArgumentParser
import os
from os.path import join

import torch
from soundfile import write
from torchaudio import load
from tqdm import tqdm

from sgmse.model import ScoreModel
from sgmse.util.other import ensure_dir, pad_spec

In [ ]:
# noisy_dir = join(args.test_dir, 'noisy/')
noisy_dir = '/content/samples'
# checkpoint_file = args.ckpt
checkpoint_file = 'train_vb.ckpt'
# corrector_cls = args.corrector
corrector_cls = 'ald'

# target_dir = args.enhanced_dir
target_dir = '/content/result'
ensure_dir(target_dir)

# Settings
sr = 16000
# snr = args.snr
snr = 0.5
# N = args.N
N = 30
# corrector_steps = args.corrector_steps
corrector_steps = 1

# Load score model 
model = ScoreModel.load_from_checkpoint(checkpoint_file, base_dir='', batch_size=16, num_workers=0, kwargs=dict(gpu=False))
model.eval(no_ema=False)
model.cuda()

noisy_files = sorted(glob.glob('{}/*.wav'.format(noisy_dir)))

for noisy_file in tqdm(noisy_files):
    filename = noisy_file.split('/')[-1]
    
    # Load wav
    y, _ = load(noisy_file) 
    T_orig = y.size(1)   

    # Normalize
    norm_factor = y.abs().max()
    y = y / norm_factor
    
    # Prepare DNN input
    Y = torch.unsqueeze(model._forward_transform(model._stft(y.cuda())), 0)
    Y = pad_spec(Y)
    
    # Reverse sampling
    sampler = model.get_pc_sampler(
        'reverse_diffusion', corrector_cls, Y.cuda(), N=N, 
        corrector_steps=corrector_steps, snr=snr)
    sample, _ = sampler()
    
    # Backward transform in time domain
    x_hat = model.to_audio(sample.squeeze(), T_orig)

    # Renormalize
    x_hat = x_hat * norm_factor

    # Write enhanced wav file
    write(join(target_dir, filename), x_hat.cpu().numpy(), 16000)

In [5]:
!ls /content

gdrive		      result   sample_data  sgmse	   wsj.cpkt
model_vb_epo115.cpkt  results  samples	    train_vb.ckpt


In [6]:
!ls

model_vb_epo115.cpkt  sample_data  sgmse  wsj.cpkt


In [2]:
import IPython

IPython.display.display(IPython.display.Audio('samples/sample_0.wav'))
IPython.display.display(IPython.display.Audio('samples/sample_1.wav'))
IPython.display.display(IPython.display.Audio('samples/sample_2.wav'))